One way to test for a warming climate is to measure the number of days above and below average over a time period. If temperature were not increasing, a reasonable conclusion is that the number of days above and below average would be roughly equal over long enough time periods. By contrast, a warming climate would see more days above average than below. We can calculate this by checking each day against a 30 year rolling average for that particular day in year and measuring the deviation from this number.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
whitestown = pd.read_csv('C:\csv\whitestownweather.csv')

cols = ['DATE', 'TMAX', 'TMIN']

whitestown = whitestown[cols]
whitestown['DATE'] = pd.to_datetime(whitestown['DATE'])
whitestown['YEAR'] = whitestown['DATE'].dt.year
whitestown['DOY'] = whitestown['DATE'].dt.dayofyear # day of year
whitestown.head()

,DATE,TMAX,TMIN,YEAR,DOY
0,1901-01-01,20.0,-4.0,1901,1
1,1901-01-02,21.0,-12.0,1901,2
2,1901-01-03,28.0,-6.0,1901,3
3,1901-01-04,40.0,3.0,1901,4
4,1901-01-05,35.0,18.0,1901,5


Let's start with 1969. We will slice this data out but still use the original dateframe to calculate with (because we need 1938-1968).

In [9]:
whitestown_slice = whitestown[whitestown['YEAR'] > 1968]
whitestown_slice.head()

,DATE,TMAX,TMIN,YEAR,DOY
24835,1969-01-01,12.0,-2.0,1969,1
24836,1969-01-02,17.0,-2.0,1969,2
24837,1969-01-03,29.0,10.0,1969,3
24838,1969-01-04,20.0,-4.0,1969,4
24839,1969-01-05,11.0,-5.0,1969,5


In [18]:
anom_list = []

for index, row in whitestown_slice.iterrows():
    start = row[3] - 30
    data_slice = whitestown[whitestown['DOY'] == row[4]]
    data_slice = data_slice[data_slice['YEAR'] > start]
    data_slice = data_slice[data_slice['YEAR'] < row[3]]
    
    average = data_slice['TMAX'].mean()
    anomaly = row[1] - average
    
    anom_list.append(anomaly)
    
whitestown_slice['ANOMALY'] = anom_list

C:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
whitestown_pos = whitestown_slice[whitestown_slice['ANOMALY'] > 0]
whitestown_neg = whitestown_slice[whitestown_slice['ANOMALY'] < 0]

In [20]:
whitestown_pos.shape[0]

9244

In [21]:
whitestown_neg.shape[0]

8930

In [17]:
whitestown_slice['ANOMALY'].head(100)

24835   -14.724138
24836   -14.724138
24837   -14.724138
24838   -14.724138
24839   -14.724138
24840   -14.724138
24841   -14.724138
24842   -14.724138
24843   -14.724138
24844   -14.724138
24845   -14.724138
24846   -14.724138
24847   -14.724138
24848   -14.724138
24849   -14.724138
24850   -14.724138
24851   -14.724138
24852   -14.724138
24853   -14.724138
24854   -14.724138
24855   -14.724138
24856   -14.724138
24857   -14.724138
24858   -14.724138
24859   -14.724138
24860   -14.724138
24861   -14.724138
24862   -14.724138
24863   -14.724138
24864   -14.724138
           ...    
24905   -14.724138
24906   -14.724138
24907   -14.724138
24908   -14.724138
24909   -14.724138
24910   -14.724138
24911   -14.724138
24912   -14.724138
24913   -14.724138
24914   -14.724138
24915   -14.724138
24916   -14.724138
24917   -14.724138
24918   -14.724138
24919   -14.724138
24920   -14.724138
24921   -14.724138
24922   -14.724138
24923   -14.724138
24924   -14.724138
24925   -14.724138
24926   -14.